In [ ]:
%pip install tensorflow==2.16.1

%pip install deepxde

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 130.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: ml-dtypes
    F

In [ ]:
import sys


# Suppress TensorFlow logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Reduces TensorFlow verbosity
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations if needed

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

import numpy as np
import matplotlib
matplotlib.use('pdf')
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs

import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import deepxde as dde
import json


# Set the random seed for NumPy
np.random.seed(1234)
# Set the random seed for TensorFlow
tf.set_random_seed(1234)

# Check GPU availability and print in red
gpu_available = tf.config.list_physical_devices('GPU')
print("\033[91mGPU Available: ", gpu_available, "\033[0m")
time.sleep(2)

############################
D2=0.0
D3=1/16

R2=0.0
R3=1/16
############################

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub, X_u_train):

        X0 = np.concatenate((x0, 0 * x0), 1)  # (x0, 0)
        X_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
        X_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)

        self.lb = lb
        self.ub = ub
        self.X_u_train = X_u_train

        self.x0 = X0[:, 0:1]
        self.t0 = X0[:, 1:2]

        self.x_lb = X_lb[:, 0:1]
        self.t_lb = X_lb[:, 1:2]

        self.x_ub = X_ub[:, 0:1]
        self.t_ub = X_ub[:, 1:2]

        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]

        self.u0 = u0
        self.v0 = v0

        # Initialize NNs
        self.layers = layers
        self.weights, self.biases = self.initialize_NN(layers)

        # tf Placeholders
        self.x0_tf = tf.placeholder(tf.float32, shape=[None, self.x0.shape[1]])
        self.t0_tf = tf.placeholder(tf.float32, shape=[None, self.t0.shape[1]])

        self.u0_tf = tf.placeholder(tf.float32, shape=[None, self.u0.shape[1]])
        self.v0_tf = tf.placeholder(tf.float32, shape=[None, self.v0.shape[1]])

        self.x_lb_tf = tf.placeholder(tf.float32, shape=[None, self.x_lb.shape[1]])
        self.t_lb_tf = tf.placeholder(tf.float32, shape=[None, self.t_lb.shape[1]])

        self.x_ub_tf = tf.placeholder(tf.float32, shape=[None, self.x_ub.shape[1]])
        self.t_ub_tf = tf.placeholder(tf.float32, shape=[None, self.t_ub.shape[1]])

        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])

        # tf Graphs
        self.u0_pred, self.v0_pred, _, _ = self.net_uv(self.x0_tf, self.t0_tf)
        self.u_lb_pred, self.v_lb_pred, self.u_x_lb_pred, self.v_x_lb_pred = self.net_uv(self.x_lb_tf, self.t_lb_tf)
        self.u_ub_pred, self.v_ub_pred, self.u_x_ub_pred, self.v_x_ub_pred = self.net_uv(self.x_ub_tf, self.t_ub_tf)
        self.f_u_pred, self.f_v_pred = self.net_f_uv(self.x_f_tf, self.t_f_tf)

        # Loss
        self.loss = tf.reduce_mean(tf.square(self.u0_tf - self.u0_pred)) + \
                    tf.reduce_mean(tf.square(self.v0_tf - self.v0_pred)) + \
                    tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.v_lb_pred - self.v_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.u_x_lb_pred - self.u_x_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.v_x_lb_pred - self.v_x_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.f_u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_v_pred))

        # Optimizers using TensorFlow 2.x
        self.optimizer = dde.optimizers.tensorflow_compat_v1.scipy_optimizer.ScipyOptimizerInterface(
            self.loss, method='L-BFGS-B',
            options={'maxiter': 10000, 'maxfun': 10000, 'maxcor': 50, 'maxls': 50, 'ftol': 1.0 * np.finfo(float).eps})

        #########################################################
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        self.gradients = tf.gradients(self.loss, self.weights)

        # tf session
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))

        init = tf.global_variables_initializer()
        self.sess.run(init)

        self.losses_adam = []
        self.losses_bfgs = []
        self.iteration_counter = 0

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = tf.Variable(tf.random_normal([layers[l], layers[l + 1]], stddev=np.sqrt(1 / layers[l])),
                            dtype=tf.float32)
            b = tf.Variable(tf.random_normal([1, layers[l + 1]]), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1

        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(0, num_layers - 2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    def net_uv(self, x, t):
        X = tf.concat([x, t], 1)

        uv = self.neural_net(X, self.weights, self.biases)
        u = uv[:, 0:1]
        v = uv[:, 1:2]

        u_x = tf.gradients(u, x)[0]
        v_x = tf.gradients(v, x)[0]

        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        u, v, u_x, v_x = self.net_uv(x, t)

        u_t = tf.gradients(u, t)[0]
        u_xx = tf.gradients(u_x, x)[0]


        v_t = tf.gradients(v, t)[0]
        v_xx = tf.gradients(v_x, x)[0]


        #   style
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u



        return f_u, f_v

    def callback(self, loss):
        self.iteration_counter += 1
        if self.iteration_counter % 100 == 0:
          print(f'Iteration {self.iteration_counter}: Loss L-BFGS-B = {loss:.5e}')

    def train(self, nIter):

        tf_dict = {self.x0_tf: self.x0, self.t0_tf: self.t0,
                   self.u0_tf: self.u0, self.v0_tf: self.v0,
                   self.x_lb_tf: self.x_lb, self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub, self.t_ub_tf: self.t_ub,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}

        start_time = time.time()
        for it in range(nIter):
            _, loss_value, gradients = self.sess.run([self.train_op_Adam, self.loss, self.gradients], tf_dict)
            self.losses_adam.append(loss_value)

            # Print
            if it % 100 == 0:
                elapsed = time.time() - start_time
                print(f'Iteration {it}, Loss Adam: {loss_value}')
                start_time = time.time()

        self.optimizer.minimize(self.sess, feed_dict=tf_dict, fetches=[self.loss], loss_callback=self.callback)

    def predict(self, X_star):

            tf_dict = {self.x0_tf: X_star[:, 0:1], self.t0_tf: X_star[:, 1:2]}
            u_star = self.sess.run(self.u0_pred, tf_dict)
            v_star = self.sess.run(self.v0_pred, tf_dict)

            tf_dict = {self.x_f_tf: X_star[:, 0:1], self.t_f_tf: X_star[:, 1:2]}

            f_u_star = self.sess.run(self.f_u_pred, tf_dict)
            f_v_star = self.sess.run(self.f_v_pred, tf_dict)

            return u_star, v_star, f_u_star, f_v_star

if __name__ == "__main__":
    noise = 0.0

    # Training domain
    lb = np.array([-5.0, 0.0])
    ub = np.array([ 5.0, np.pi/2])

    N0 = 100
    N_b = 100
    N_f = 20000
    layers = [2, 40, 40, 40, 2]

    data = scipy.io.loadmat('NLS.mat')


    t = data['tt'].flatten()[:, None]
    x = data['x'].flatten()[:, None]
    Exact = data['uu']

    Exact_u = np.real(Exact)
    Exact_v = np.imag(Exact)
    Exact_h = np.sqrt(Exact_u ** 2 + Exact_v ** 2)

    X, T = np.meshgrid(x, t)

    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    u_star = Exact_u.T.flatten()[:, None]
    v_star = Exact_v.T.flatten()[:, None]
    h_star = Exact_h.T.flatten()[:, None]

    idx_x = np.random.choice(x.shape[0], N0, replace=False)
    x0 = x[idx_x, :]
    u0 = Exact_u[idx_x, 0:1]
    v0 = Exact_v[idx_x, 0:1]

    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t, :]

    X_f = lb + (ub - lb) * lhs(2, N_f)

    X0 = np.concatenate((x0, 0 * x0), 1)  # (x0, 0)
    X_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
    X_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)
    X_u_train = np.vstack([X0, X_lb, X_ub])

    model = PhysicsInformedNN(x0, u0, v0, tb, X_f, layers, lb, ub, X_u_train)

    start_time = time.time()

    model.train(10000)



    elapsed = time.time() - start_time
    print('Training time: %.4f' % (elapsed))

    u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star)
    h_pred = np.sqrt(u_pred ** 2 + v_pred ** 2)

    error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    error_v = np.linalg.norm(v_star - v_pred, 2) / np.linalg.norm(v_star, 2)
    error_h = np.linalg.norm(h_star - h_pred, 2) / np.linalg.norm(h_star, 2)
    print('Error u: %e' % (error_u))
    print('Error v: %e' % (error_v))
    print('Error h: %e' % (error_h))


    U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
    V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
    H_pred = griddata(X_star, h_pred.flatten(), (X, T), method='cubic')

    U_star = griddata(X_star, u_star.flatten(), (X, T), method='cubic')
    V_star = griddata(X_star, v_star.flatten(), (X, T), method='cubic')
    H_star = griddata(X_star, h_star.flatten(), (X, T), method='cubic')

    FU_pred = griddata(X_star, f_u_pred.flatten(), (X, T), method='cubic')
    FV_pred = griddata(X_star, f_v_pred.flatten(), (X, T), method='cubic')



########################################################################
# Plotting the stacked time slices with prediction and exact solutions
plt.figure(figsize=(10, 8))
x_min, x_max = x.min(), x.max()
cut_indices = np.linspace(0, len(t) - 1, 9, dtype=int)
y_min, y_max = -0.1, (len(cut_indices) + 1) * 2.0

for i, idx in enumerate(cut_indices):
    vertical_offset = i * 2.0  # Adjust the spacing between lines as needed
    plt.plot(x, Exact_h[:, idx] + vertical_offset, 'b-', linewidth=1.5, label=f'$t = {t[idx, 0]:.2f}$' if i == 0 else "")
    plt.plot(x, H_pred[idx, :] + vertical_offset, 'r--', linewidth=1.5)
    plt.text(x_min - 1, vertical_offset, f'$t = {t[idx, 0]:.2f}$', fontsize=10, verticalalignment='center')
plt.xlabel('$x$', fontsize=14)
plt.ylabel('')
plt.xlim([x_min, x_max])
plt.ylim([y_min, y_max])
plt.gca().set_yticks([])  # Remove y-axis ticks
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(['Exact', 'Prediction'], loc='upper right', fontsize=10)
plt.tight_layout()
plt.savefig('time.pdf', dpi=300)


############################ Plotting ###############################
# Plotting the results
fig, ax = plt.subplots(figsize=(10, 8))

# We do NOT transpose if the first dimension is t, second is x.
h_img = ax.imshow(H_pred,
                  extent=[lb[0], ub[0], lb[1], ub[1]],  # [x_min, x_max, t_min, t_max]
                  origin='lower',
                  aspect='auto',
                  cmap='YlGnBu')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h_img, cax=cax, label='|Q(x, t)|')

# 3) Plot training points
ax.plot(X_u_train[:, 0], X_u_train[:, 1],
        'kx', label='Data Points (%d)' % (X_u_train.shape[0]),
        markersize=4, clip_on=False)
ax.plot(X_f[:, 0], X_f[:, 1],
        'r.', label='Collocation Points (%d)' % (X_f.shape[0]),
        markersize=2)
ax.legend(loc='upper right', fontsize=10, framealpha=0.8)
ax.set_xlabel("x", fontsize=14)
ax.set_ylabel("t", fontsize=14)
plt.tight_layout()
plt.savefig('data.pdf', dpi=150)
plt.show()


#############################################################################
plt.figure(figsize=(10, 8))

# Plotting exact u(t, x)
plt.subplot(3, 3, 1)
plt.pcolor(T, X, U_star, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Exact u(t, x)')
plt.tight_layout()

# Plotting exact v(t, x)
plt.subplot(3, 3, 2)
plt.pcolor(T, X, V_star, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Exact v(t, x)')
plt.tight_layout()

# Plotting exact h(t, x)
plt.subplot(3, 3, 3)
plt.pcolor(T, X, H_star, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Exact h(t, x)')
plt.tight_layout()

# Plotting predicted u(t, x)
plt.subplot(3, 3, 4)
plt.pcolor(T, X, U_pred, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Predicted u(t, x)')
plt.tight_layout()

# Plotting predicted v(t, x)
plt.subplot(3, 3, 5)
plt.pcolor(T, X, V_pred, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Predicted v(t, x)')
plt.tight_layout()

# Plotting predicted h(t, x)
plt.subplot(3, 3, 6)
plt.pcolor(T, X, H_pred, cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Predicted h(t, x)')
plt.tight_layout()

# Plotting absolute error for u(t, x)
plt.subplot(3, 3, 7)
plt.pcolor(T, X, np.abs(U_star - U_pred), cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Absolute error for u(t, x)')
plt.text(0.1, 0.9, f'Error u: {error_u:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
plt.tight_layout()

# Plotting absolute error for v(t, x)
plt.subplot(3, 3, 8)
plt.pcolor(T, X, np.abs(V_star - V_pred), cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Absolute error for v(t, x)')
plt.text(0.1, 0.9, f'Error v: {error_v:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
plt.tight_layout()

# Plotting absolute error for h(t, x)
plt.subplot(3, 3, 9)
plt.pcolor(T, X, np.abs(H_star - H_pred), cmap='gray')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title('Absolute error for h(t, x)')
plt.text(0.1, 0.9, f'Error h: {error_h:.3e}', color='white', fontsize=10, transform=plt.gca().transAxes)
plt.tight_layout()
plt.savefig('U_V_H_errors.pdf', dpi=25, bbox_inches='tight')


np.savez_compressed('PINN_results.npz',
     x=X[0, :],                  # x-axis
     t=T[:, 0],                  # t-axis
     X=X,
     T=T,
     Exact_h = Exact_h,
     U_star=U_star,
     V_star=V_star,
     H_star=H_star,
     U_pred=U_pred,
     V_pred=V_pred,
     H_pred=H_pred,
     X_u_train=X_u_train,
     X_f=X_f,
     lb=lb,
     ub=ub,
     error_u=error_u,
     error_v=error_v,
     error_h=error_h
)


metadata = {
     'layers': layers,
     'N0': int(N0),
     'Nb': int(N_b),
     'Nf': int(N_f),
     'lb': lb.tolist(),
     'ub': ub.tolist(),
     'error_u': float(error_u),
     'error_v': float(error_v),
     'error_h': float(error_h),
     'training_time_sec': float(elapsed)
    }

with open('PINN_metadata.json', 'w') as f:
      json.dump(metadata, f, indent=4)

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

Iteration 0, Loss Adam: 16.128950119018555
Iteration 100, Loss Adam: 0.42226845026016235
Iteration 200, Loss Adam: 0.36647650599479675
Iteration 300, Loss Adam: 0.29819557070732117
Iteration 400, Loss Adam: 0.2187994420528412
Iteration 500, Loss Adam: 0.15566936135292053
Iteration 600, Loss Adam: 0.12818194925785065
Iteration 700, Loss Adam: 0.11291806399822235
Iteration 800, Loss Adam: 0.10041521489620209
Iteration 900, Loss Adam: 0.08914681524038315
Iteration 1000, Loss Adam: 0.0805090069770813
Iteration 1100, Loss Adam: 0.07425364851951599
Iteration 1200, Loss Adam: 0.069801926612854
Iteration 1300, Loss Adam: 0.0663832500576973
Iteration 1400, Loss Adam: 0.06348473578691483
Iteration 1500, Loss Adam: 0.060916826128959656
Iteration 1600, Loss Adam: 0.0586026124

In [ ]:
!pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=a7f0da8d751d9f57e074121ef005d8552f3a2e4fc943af7006bfa9d69e375401
  Stored in directory: /root/.cache/pip/wheels/96/b9/5d/1138ea8c8f212bce6e97ae58847b7cc323145b3277f2129e2b
Successfully built pyDOE
